In [1]:
import logging
import numpy as np
import scipy.io
import matplotlib.pyplot as plt


# from examples.us_bmode_linear_transducer.example_utils import download_if_does_not_exist
from kwave.data import Vector
from kwave.kgrid import kWaveGrid
from kwave.kmedium import kWaveMedium
from kwave.kspaceFirstOrder3D import kspaceFirstOrder3D
from kwave.ktransducer import NotATransducer, kWaveTransducerSimple
from kwave.options.simulation_execution_options import SimulationExecutionOptions
from kwave.options.simulation_options import SimulationOptions
from kwave.utils.dotdictionary import dotdict
from kwave.utils.signals import tone_burst, get_win
from kwave.utils.filters import gaussian_filter
from kwave.utils.conversion import db2neper
from kwave.reconstruction.tools import log_compression
from kwave.reconstruction.beamform import envelope_detection
from kwave.utils.mapgen import make_ball
from utils_ex.artifacts import get_phantom_data_circle, get_3d_circle, visualize_maps
from utils_ex.sim_params import Params

%load_ext autoreload
%autoreload 2


In [2]:
args = Params()
grid_size_points = args.grid_size_points
grid_spacing_meters = args.grid_spacing_meters
c0 = args.c0
rho0 = args.rho0
tone_burst_freq = args.tone_burst_freq
tone_burst_cycles = args.tone_burst_cycles
SOME_TIME_CONSTANT = args.SOME_TIME_CONSTANT
source_strength = args.source_strength
kgrid = kWaveGrid(grid_size_points, grid_spacing_meters)
t_end = (grid_size_points.x * grid_spacing_meters.x) * SOME_TIME_CONSTANT / c0  # [s]
kgrid.makeTime(c0, t_end=t_end)

input_signal = tone_burst(1 / kgrid.dt, tone_burst_freq, tone_burst_cycles)
input_signal = (source_strength / (c0 * rho0)) * input_signal

medium = kWaveMedium(
    sound_speed=None,  # will be set later
    alpha_coeff=args.alpha_coeff,
    alpha_power=args.alpha_power,
    BonA=args.BonA,
)

transducer = dotdict()
transducer.number_elements = 32  # total number of transducer elements
transducer.element_width = 2  # width of each element [grid points/voxels]
transducer.element_length = 24  # length of each element [grid points/voxels]
transducer.element_spacing = (
    0  # spacing (kerf  width) between the elements [grid points/voxels]
)
transducer.radius = float("inf")  # radius of curvature of the transducer [m]

# calculate the width of the transducer in grid points
transducer_width = (
    transducer.number_elements * transducer.element_width
    + (transducer.number_elements - 1) * transducer.element_spacing
)

# use this to position the transducer in the middle of the computational grid
transducer.position = np.round(
    [
        1,
        grid_size_points.y / 2 - transducer_width / 2,
        grid_size_points.z / 2 - transducer.element_length / 2,
    ]
)
transducer = kWaveTransducerSimple(kgrid, **transducer)


not_transducer = dotdict()
not_transducer.sound_speed = c0  # sound speed [m/s]
not_transducer.focus_distance = 20e-3  # focus distance [m]
not_transducer.elevation_focus_distance = (
    19e-3  # focus distance in the elevation plane [m]
)
not_transducer.steering_angle = 0  # steering angle [degrees]
not_transducer.transmit_apodization = "Hanning"
not_transducer.receive_apodization = "Rectangular"
not_transducer.active_elements = np.ones((transducer.number_elements, 1))
not_transducer.input_signal = input_signal

not_transducer = NotATransducer(transducer, kgrid, **not_transducer)


In [3]:
!pwd

/c/Users/CMME3/Documents/GitHub/k-wave-python/examples/us_bmode_linear_transducer


In [4]:
logging.log(logging.INFO, "Fetching phantom data...")
# download_if_does_not_exist(PHANTOM_DATA_GDRIVE_ID, PHANTOM_DATA_PATH)
params_dict = args.__dict__
phantom = get_phantom_data_circle(
                                kgrid=kgrid,
                                params = params_dict,
                                debug = True
                                )

sound_speed_map = phantom["sound_speed_map"]
density_map = phantom["density_map"]

In [5]:
from utils_ex.sim_params import Params
from utils_ex.helper import (
    get_transducer,
    make_grid,
    get_input_signal,
    solve_kspace_problem,
    process_simulation_data
)
from utils_ex.artifacts import get_phantom_data_circle

from kwave.kmedium import kWaveMedium  # type: ignore
print("Imported all modules")

def run_simulation(args: Params):
    print("Running Simulation")
    # define the grid
    grid_size_points = args.grid_size_points
    grid_spacing_meters = args.grid_spacing_meters
    c0 = args.c0
    rho0 = args.rho0
    tone_burst_freq = args.tone_burst_freq
    tone_burst_cycles = args.tone_burst_cycles
    SOME_TIME_CONSTANT = args.SOME_TIME_CONSTANT
    source_strength = args.source_strength
    # create the computational grid
    kgrid = make_grid(grid_size_points, grid_spacing_meters, c0, SOME_TIME_CONSTANT)
    # create the time array and input signal
    input_signal = get_input_signal(kgrid, tone_burst_freq, tone_burst_cycles, source_strength, c0, rho0)
    # define the transducer properties
    not_transducer,transducer = get_transducer(grid_size_points, input_signal, kgrid, c0)
    # define the medium properties
    medium = kWaveMedium(
        sound_speed=None,  # will be set later
        alpha_coeff=args.alpha_coeff,
        alpha_power=args.alpha_power,
        BonA=args.BonA,
    )
    # get the phantom
    phantom = get_phantom_data_circle(
                                    kgrid=kgrid,
                                    params = args.__dict__,
                                    debug = True
                                    )
    # get the phantom data and set the medium properties
    medium.sound_speed = phantom["sound_speed_map"]
    medium.density_map = phantom["density_map"]
    # break
    # run the simulation to get scan lines
    # scan_lines = solve_kspace_problem(
    #     kgrid=kgrid,
    #     medium=medium,
    #     args=args,
    #     sound_speed_map=phantom["sound_speed_map"],
    #     density_map=phantom["density_map"],
    #     grid_size_points=grid_size_points,
    #     transducer=transducer,
    #     not_transducer=not_transducer,        
    # )
    # # Process the scan lines using RF Signal Processing
    # processed_central_scan_line = process_simulation_data(kgrid=kgrid,
    #                         medium=medium,
    #                         input_signal=input_signal,
    #                         scan_lines=scan_lines,
    #                         tone_burst_freq=tone_burst_freq,
    #                         c0=c0,
    #                         args=args)
    # return processed_central_scan_line

print("Hello World")
args = Params()  # loading the parameters
run_simulation(args = args)

Imported all modules
Hello World
Running Simulation


In [8]:
a

NameError: name 'a' is not defined